# Preprocessing Data

1. Convert Different Language Comments into English

In [2]:
%pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.0 MB/s eta 0:00:00a 0:00:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=20079b31a08c28e849ff1d4ade94a7457990d72747bd2583cfe90895f311826e
  Stored in directory: /home/skywalker/.cache/pip/wheels/3c/bc/9d/266e27289b9019680d65d9b608c37bff1eff565b001c977ec5
Successfully built langid
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import tqdm

In [4]:
from deep_translator import GoogleTranslator
import langid

def translate(text):
  if ((not pd.isna(text)) and langid.classify(text) != 'en'):
    translator = GoogleTranslator(source='auto', target='en')
    return translator.translate(text)
  else:
    return text


In [6]:
fvc = pd.read_csv("Datasets/Updated_FVC.csv")
mvd = pd.read_csv("Datasets/Updated_MVD.csv")
bolly = pd.read_csv("Datasets/Updated_BollyBAIT.csv")
combined = pd.read_csv("Datasets/Updated_Combined.csv")

In [7]:
fvc

,video_id,LABEL,title,description,likes,comments,views
0,Y7MJlmbnLQA,FAKE,بنت سقطت في حمم بركانيه 🙀,اللهم اجرنا من عذاب جهنم,66,42,18201
1,AEPvSo8bE2I,FAKE,Michel & Sven - Der Tischdeckentrick - Teil 4 ...,http://www.facebook.com/michelundsven\nhttp://...,92529,0,25398348
2,CXOU10WBogs,FAKE,Backflip and almost hit by car!,Jukin Media Verified (Original)\n* For licensi...,384,26,169870
3,ZoHAf7VtIhA,FAKE,Shark Attack - Cam Girl Bitten off Florida Coast,Camsoda model Molly Cavalli was bitten while b...,8608,1930,17784032
4,RXF5yZZe8nQ,FAKE,American Airline plane catches fire on runway,Passengers run for survival as a plane catches...,36,24,4682
...,...,...,...,...,...,...,...
242,XvMoYEUVcB4,REAL,Dolphin Seen at Mahalaxmi Mumbai,Today people seen Dolphins In Mahalaxmi Mumbai...,11,0,971
243,lYFpga7kJxE,REAL,Angry mob throwing stones on mumbai local trai...,Angry mob throwing stones on mumbai local trai...,3275,418,2439095
244,WNjFFTQsIAg,REAL,Woman carted off by Muslim horde into subway s...,"No idea where this was filmed, some are saying...",43,22,4943
245,x76VEPXYaI0,REAL,GoPro: Backflip Over 72ft Canyon - Kelly McGar...,Shot 100% on the HD HERO3+® camera from ‪http:...,2561286,58142,224438799


In [10]:
fvc.loc[13,'description']

'إعصار,\nإعصار البحر,\nارما,\nإرما,\nاعصار ارما,\nإعصار إرما,\nسرعة الرياح فى الاعصار,\nسرعة رياح اعصار ارما,\nإعصار النار,\nإعصار اليابان,\nإعصار اليابان 2017,\nإعصار بغداد,\nإعصار تسونامي,\nإعصار فيه نار,\nإعصار قوي,\nإعصار نان مادول,\nاخر اعصار ضرب امريكا,\nاعصار,\nاعصار 2016,\nاعصار 2016 امريكا,\nاعصار 2017,\nاعصار ارما,\nاعصار اسطنبول,\nاعصار اسماك القرش 2,\nاعصار البحر,\nاعصار الفلبين,\nاعصار الفلبين 2016,\nاعصار القرش,\nاعصار النار,\nاعصار اليابان,\nاعصار اليابان 2017,\nاعصار امريكا,\nاعصار اولاد سي حمد,\nاعصار ايرما,\nاعصار باتريسيا,\nاعصار بالي,\nاعصار بحر,\nاعصار بحر العرب,\nاعصار بحري,\nاعصار بداخلي,\nاعصار بداخلي 3,\nاعصار بغداد,\nاعصار بنجلاديش,\nاعصار تاتسوماكي,\nاعصار تالاس,\nاعصار تايوان,\nاعصار تايوان 2016,\nاعصار ترابي,\nاعصار تركيا,\nاعصار تركيا اليوم,\nاعصار تركيا امس,\nاعصار تسونامي,\nاعصار تسونامي خالد الراشد,\nاعصار تسونامي في اليابان,\nاعصار تشابالا,\nاعصار تشابالا في اليمن,\nاعصار ثلجي,\nاعصار جازان,\nاعصار جده,\nاعصار جميل,\nاعصار جوبلين,\nاعصار جونو,\nاعصار ج

In [13]:
not_translated=[]

In [18]:

def dataframe_translate(df,columns):
    df1=df
    with tqdm.tqdm(total=len(df)*len(columns), desc="Translating") as progress_bar:
        for col in columns:
            for i in range(len(df)):
                try:
                    df1.loc[i,col]=translate(df.loc[i,col])
                    if(df1.loc[i,col]==df.loc[i,col]):
                        not_translated.append(i)
                except Exception as e:
                    print(f"Error translating row {i + 1}: {e}")
                    not_translated.append(i)
                    
                progress_bar.update(1)
    return df1


In [19]:
df = fvc
df=dataframe_translate(df,['title','description'])
df

Translating:  22%|██▏       | 107/494 [02:59<12:04,  1.87s/it]

In [16]:
%pip install mtranslate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3672 sha256=d63ae7a23fa9fda4943604ee810b31b3ebc27d94637693f2068a5d9d981eb512
  Stored in directory: /home/skywalker/.cache/pip/wheels/04/5b/83/c25d4a866e5f15021667de75eebe8e9aec1b673237a44c4d0a
Successfully built mtranslate
Note: you may need to restart the kernel to use updated packages.


In [17]:
from deep_translator import GoogleTranslator
from googletrans import Translator as GoogleTransTranslator
from mtranslate import translate as MTranslateTranslator
from translate import Translator as TranslateTranslator
import langid

def translate_with_deep_translator(text):
    translator = GoogleTranslator(source='auto', target='en')
    return translator.translate(text)

def translate_with_googletrans(text):
    translator = GoogleTransTranslator()
    translation = translator.translate(text, src='auto', dest='en')
    return translation.text

def translate_with_mtranslate(text):
    return MTranslateTranslator(text, 'en')

def translate_with_translate(text):
    translator = TranslateTranslator(to_lang='en')
    return translator.translate(text).text  # Access the text attribute

def translate_with_langid(text):
    _, lang = langid.classify(text)
    if lang != 'en':
        return translate_with_translate(text)
    return text

text_to_translate = "بنت سقطت في حمم بركانيه 🙀"

# Deep Translator
result_deep_translator = translate_with_deep_translator(text_to_translate)
print(f"Deep Translator: {result_deep_translator}")

# Googletrans
result_googletrans = translate_with_googletrans(text_to_translate)
print(f"Googletrans: {result_googletrans}")

# mtranslate
result_mtranslate = translate_with_mtranslate(text_to_translate)
print(f"mtranslate: {result_mtranslate}")

# translate
result_translate = translate_with_translate(text_to_translate)
print(f"translate: {result_translate}")

# langid
result_langid = translate_with_langid(text_to_translate)
print(f"langid: {result_langid}")


Deep Translator: A girl fell into lava 🙀
Googletrans: A girl fell into lava 🙀
mtranslate: A girl fell into lava 🙀


AttributeError: 'str' object has no attribute 'text'